In [261]:
import pandas as pd, os, numpy as np, datetime, math

In [45]:
os.chdir('/Users/simonaaksman/Desktop/Git/datadive_201608_ci')
data = './data/processed/'
os.chdir(data)

In [144]:
marin = pd.read_csv('Marin_Merged.csv')
team = pd.read_csv('Terrestrial_Vertebrate_Cols_Edited.csv')

In [56]:
marin.columns

Index([u'Unnamed: 0', u'Project.ID', u'Deployment.ID', u'Image.ID',
       u'Location', u'Photo.Type', u'Photo.Type.Identified.by',
       u'Genus.Species', u'Uncertainty', u'IUCN.Identification.Number',
       u'Date_Time.Captured', u'Age', u'Sex', u'Individual.ID', u'Count',
       u'Animal.recognizable..Y.N.', u'individual.Animal.notes',
       u'Project.ID.x', u'Event', u'Array.Name', u'Deployment.Location.ID',
       u'Longitude.Resolution', u'Latitude.Resolution',
       u'Camera.Deployment.Begin.Date', u'Camera.Deployment.End.Date',
       u'Bait.Type', u'Bait.Description', u'Feature.Type',
       u'Feature.Type.methodology', u'Camera.ID', u'Quiet.Period.Setting',
       u'Restriction.on.access', u'Camera.Failure.Details', u'Project.ID.y',
       u'Make', u'Model', u'Serial.Number', u'Year.Purchased', u'Genus',
       u'Species'],
      dtype='object')

In [55]:
team.columns

Index([u'Unnamed: 0', u'ID', u'Project.ID', u'Deployment.Location.ID',
       u'Latitude.Resolution', u'Longitude.Resolution', u'Sampling.Period',
       u'Photo.Type', u'Photo.Date', u'Photo.Time', u'Image.ID',
       u'Photo.Notes', u'Location', u'Class', u'Order', u'Family', u'Genus',
       u'Species', u'Count', u'Photo.Type.Identified.by',
       u'Camera.Serial.Number', u'Memory.Card.Serial.Number', u'Camera.Notes',
       u'Camera.Deployment.Begin.Date', u'Camera.Deployment.End.Date',
       u'Person.setting.up.the.Camera', u'Person.picking.up.the.Camera',
       u'Camera.Make', u'Camera.Model', u'Flash', u'Exposure.Time',
       u'Sequence.Info', u'Triplet.Number', u'Moon.Phase', u'Temperature',
       u'Camera.Firmware.Version', u'Protocol.Version',
       u'Data.Set.Creator.Institutions', u'Data.Set.Creator.Scientist',
       u'Data.Set.Contact', u'Data.Level'],
      dtype='object')

In [91]:
# Number of locations 

len(set(team['Deployment.Location.ID']))

1058

In [101]:
# Number of projects

len(set(team['Project.ID']))

17

In [104]:
# Number of unique identifiers

len(set(team['ID']))

2286513

UI Layer Columns:
    
Data Source,Project ID,Deployment Location ID,Latitude Resolution,
Longitude Resolution,Sampling Type,Sampling Period,Year,Genus,Species,Rate Of Detection

In [286]:
team_simple = team[['Project.ID',
'Location',
'Latitude.Resolution',
'Longitude.Resolution',
'Sampling.Period',
'Deployment.Location.ID',
'Camera.Deployment.Begin.Date',
'Camera.Deployment.End.Date',
'Camera.Serial.Number',
'Genus', 'Species']]


team_simple_per_location = team_simple.groupby(['Project.ID', 
                                                'Deployment.Location.ID', 
                      'Camera.Deployment.Begin.Date',
                      'Camera.Deployment.End.Date'])

In [287]:
team_simple['Sampling.Period'].value_counts()

2013.01    403933
2011.01    372250
2015.01    360680
2012.01    358518
2014.01    358011
2010.01    222527
2009.01    123783
2016.01     55029
2008.01     19124
2013.02     11994
2007.01      5869
dtype: int64

In [288]:
assert(len(team_simple_per_location['Project.ID']) == 5328)

In [289]:
team_simple_per_location = team_simple_per_location.count().reset_index()

In [290]:
team_simple_per_location['Camera.Deployment.Begin.Date_datetime'] = team_simple_per_location['Camera.Deployment.Begin.Date'].apply(lambda x: 
                                                                    datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

team_simple_per_location['Camera.Deployment.End.Date_datetime'] = team_simple_per_location['Camera.Deployment.End.Date'].apply(lambda x: 
                                                                    datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [291]:
team_simple_per_location.head()

,Project.ID,Deployment.Location.ID,Camera.Deployment.Begin.Date,Camera.Deployment.End.Date,Location,Latitude.Resolution,Longitude.Resolution,Sampling.Period,Camera.Serial.Number,Genus,Species,Camera.Deployment.Begin.Date_datetime,Camera.Deployment.End.Date_datetime
0,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2011-01-15 14:08:46,2011-02-21 12:17:23,250,251,251,251,251,250,250,2011-01-15 14:08:46,2011-02-21 12:17:23
1,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2011-12-16 10:55:02,2012-01-18 11:12:03,310,311,311,311,311,280,280,2011-12-16 10:55:02,2012-01-18 11:12:03
2,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2012-12-21 13:49:05,2013-01-20 15:08:01,580,581,581,581,581,580,580,2012-12-21 13:49:05,2013-01-20 15:08:01
3,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2013-12-21 10:46:02,2014-01-20 11:27:01,520,521,521,521,521,520,520,2013-12-21 10:46:02,2014-01-20 11:27:01
4,Barro Colorado Nature Monument - Soberania Nat...,CT-BCI-1-01,2014-12-15 00:00:00,2015-01-21 12:00:00,610,611,611,611,611,610,610,2014-12-15 00:00:00,2015-01-21 12:00:00


In [292]:
team_simple_per_location['trapnights'] = (team_simple_per_location['Camera.Deployment.End.Date_datetime'] 
                                          - 
        team_simple_per_location['Camera.Deployment.Begin.Date_datetime']).astype(pd.Timedelta).apply(lambda l: l.days)

In [ ]:
for i in range(1,len(team_simple_per_location)):
    team_simple_per_location['Months'] = 

In [293]:
team_simple_per_location['Camera.Deployment.Begin.Month'] = team_simple_per_location['Camera.Deployment.Begin.Date_datetime'].apply(lambda x: 
                                                                    datetime.datetime.strptime(x, '%m'))

team_simple_per_location['Camera.Deployment.End.Month'] = team_simple_per_location['Camera.Deployment.End.Date_datime'].apply(lambda x: 
                                                                    datetime.datetime.strptime(x, '%m'))

TypeError: must be string, not Timestamp

In [268]:
team_simple_per_location['trapnights'].apply(lambda x: x.days)

AttributeError: 'numpy.timedelta64' object has no attribute 'days'